In [1]:
import re
import os
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import urllib 

# 解析

In [25]:
def recursive_parser(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
            
            content = child.text.strip()
            if len(content) == 0:
                continue
                
            if content.startswith("Medically"):
                continue
            
            if tag == 'ul' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['vitTOC']:
                    continue
            if tag == 'a' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['ddc-link-forward']:
                    continue
                    
                    
            if tag == 'h2' and 'id' in child.attrs:
                id_ = child.attrs['id']
                if id_ in ['faq']:
                    continue
                    
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['drug-subtitle', 'ddc-reviewed-by ddc-author-image', 'ddc-reviewed-by ddc-author-image-cerner']:
                    continue
            
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe', 'video', 'nav','%%fileorm']: # 加视频
                continue

            if tag == 'center':
                texts.append(content)
                continue

            if tag == 'div' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['printpreview_header', 'fdaWrapper', 'adCont', 'article-feedback_1-0', 'sources-and-citation_1-0', 'more-resources']:
                    continue
            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'drug-subtitle', 'ddc-reviewed-by ddc-author-image', 'ddc-accordion ddc-mgb-2', 'more-resources', 
                              'ddc-reviewed-by ddc-author-image-cerner']:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol', 'article']:
                # 这几类标签 直接递归处理子标签
                recursive_parser(child, texts, '')
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)

        except Exception as e:
            print(e)

In [29]:
def parser(input_path, output_path, results):
    for root, dirs, file_names in os.walk(input_path):
        """
        os.walk()
        root 所指的是当前正在遍历的这个文件夹的本身的地址
        files_names同样是 list , 内容是该文件夹中所有的文件(不包括子目录)
        函数会自动改变root的值使得遍历所有的子文件夹。
        所以返回的三元元组的个数为所有子文件夹（包括子子文件夹，子子子文件夹等等）加上1（根文件夹）
        """
        
        for file_name in file_names:
            html_file = os.path.join(root, file_name)
            html_content = open(html_file, 'r', encoding='utf-8').read()
            selector = Selector(text=html_content)
            try:
                title = selector.xpath("//h1//text()").extract()[0].strip()
            except Exception as e:
                print(e)
                print(html_file)
                continue

            texts = []
            
            soup = BeautifulSoup(html_content, 'html.parser') 
            
            item = soup.find('div',{'class':'contentBox'})

            if item is None:
                print(html_file)
                continue
            else:
                recursive_parser(item, texts, "")
                content = '\n'.join(texts)
                if content == "":
                    continue
                tmp = {
                    "title": title,
                    "content": content
                }
#                 print(tmp)
                results.append(tmp)

In [30]:
output_path = 'D:/Intern_Project/9_drugs/'
source_path = f'D:/Intern_Project/9_drugs/article_htmls'

results = []
parser(source_path, output_path, results)


In [31]:
print(len(results))

719


In [32]:
print(results[0])

{'title': 'A-200 Lice Treatment', 'content': 'A-200 Lice Treatment\nWhat is  A-200 Lice Treatment?\nPiperonyl butoxide is a chemical that stops the breakdown of pyrethrins, an insecticide chemical.\nA-200 Lice Treatment (for the skin) is a combination medicine used to treat lice.\nA-200 Lice Treatment may also be used for purposes not listed in this medication guide.\nWarnings\nFollow all directions on your medicine label and package. Tell each of your healthcare providers about all your medical conditions, allergies, and all medicines you use.\nBefore taking this medicine\nYou should not use A-200 Lice Treatment if you are allergic to it, or if you have an allergy to chrysanthemums or ragweed.\nFDA pregnancy category C. It is not known whether  A-200 Lice Treatment will harm an unborn baby. Tell your doctor if you are pregnant or plan to become pregnant while using this medication.\nIt is not known whether this medicine passes into breast milk or if it could harm a nursing baby. Tell 

In [35]:
for item in results:
    item['content'] = item['content'].split("Further information")[0]

In [36]:
print(results[0])

{'title': 'A-200 Lice Treatment', 'content': 'A-200 Lice Treatment\nWhat is  A-200 Lice Treatment?\nPiperonyl butoxide is a chemical that stops the breakdown of pyrethrins, an insecticide chemical.\nA-200 Lice Treatment (for the skin) is a combination medicine used to treat lice.\nA-200 Lice Treatment may also be used for purposes not listed in this medication guide.\nWarnings\nFollow all directions on your medicine label and package. Tell each of your healthcare providers about all your medical conditions, allergies, and all medicines you use.\nBefore taking this medicine\nYou should not use A-200 Lice Treatment if you are allergic to it, or if you have an allergy to chrysanthemums or ragweed.\nFDA pregnancy category C. It is not known whether  A-200 Lice Treatment will harm an unborn baby. Tell your doctor if you are pregnant or plan to become pregnant while using this medication.\nIt is not known whether this medicine passes into breast milk or if it could harm a nursing baby. Tell 

In [37]:
with open('D:/Intern_Project/9_drugs/parser_results.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(results, indent=1, ensure_ascii=False))